import apriltag
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import os
from utils import *

# function 1 计算Y方向的机器人世界坐标y_rob

def XY_Robot():
    # 实例化检测器并检测Apriltag
    img = updateImage()
    img = np.array(img)
#     img = cv2.imread('1.jpg')
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    options = apriltag.DetectorOptions(families='tag36h11',)
    detector = apriltag.Detector(options)
    tags = detector.detect(gray)
#     print(tags)
    cnt = 0
    while len(tags) == 0 and cnt < 5:
        img = updateImage()
        img = np.array(img)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        tags = detector.detect(gray)
        cnt += 1
        if cnt == 5:
            run_action('turn003L')
            wait_req()
            cnt = 0
    
        
#     print(tags)
    # 获取pos和corners的代码
    tag_positions = load_tag_pos() 

    # 获取pos和corners的代码
    tag_num = len(tags)
    for n, tag in enumerate(tags):
#         print("debug here::",tag.tag_id)
#         if tag.tag_id > 41:
#             continue
        # 使用tag.id访问坐标，tag_positions的定义见前面，pos的形状为[4, 3]，分别代表已知的四个顶角在空间中的x,y,z三个坐标分量
        pos = tag_positions[str(tag.tag_id)]
#         print(pos)
        # 四个顶角在图片中的像素位置(形状为[4, 2]，分别代表四个顶角的像素空间坐标) 
        corners = np.array(tag.corners)
#         print(corners)
    # 2D坐标
    ImgPoints = corners
    # 3D坐标
    WorldPoints = pos
    # 内参矩阵
    fx = 192.0513
    fy = 194.1294
    cx = 311.9018
    cy = 233.6934
    intrinsic_matrix =  np.array([[fx,0,cx],
                              [0,fy,cy],
                              [0,0,1]],dtype = np.float64)
    # 畸变系数
    distCoeffs = np.array([0.2683,-0.2053,0,0])
    # 求解R和T的值
    retval, R, T = cv2.solvePnP(WorldPoints,ImgPoints,intrinsic_matrix,distCoeffs)
    rotM = cv2.Rodrigues(R)[0]
    cameraPosition = -np.matrix(rotM).T * np.matrix(T)
    # R矩阵的第三行
    direction = np.matrix(rotM)[2]
    #
    # print("朝向::",direction)
    x_robot = cameraPosition[0]
    y_robot = cameraPosition[1]
    
    dirXY = direction[:,:2]
    normdir = dirXY / np.linalg.norm(dirXY)
    
    return y_robot,x_robot,normdir

# 写一个固定转角的函数
def rotate_to_balance(zero):
    '''
    input:
    zero,角度的基准
    output:
    None.将机器人转为X轴正方向
    '''
    _,_,dirs= XY_Robot()
    theta = angle(dirs)

    # 设置一个物理上的零点,可供机器人调参数
    distance = theta - zero

    # 设定阈值位5度
    while abs(distance) > 5:
        if distance > 0:
            run_action('turn001L')
        if distance < 0:
            run_action('turn001R')
        _,_,dirs= XY_Robot()
        theta = angle(dirs)
        distance = theta - zero
        print("Adjusting angle: theta=",theta)

    # 最后再微调一下
    if distance > 0.5:
        run_action('turn001L')
    if distance < -0.5:
        run_action('turn001R')
    print("Adjusting angle: theta=",theta)
    return 0

In [ ]:
# 获得下一个点的二维坐标，并移动
def move_to_next(next_loction_X, next_loction_Y):
    Y,X,direction = XY_Robot()
    # 转向
    move_dir = [next_loction_X-X, next_loction_Y-Y]
    norm_move_dir = move_dir / np.linalg.norm(move_dir)
    rotate_to_balance(norm_move_dir)
    # 向前走！
    run_action('Forwalk02')
    wait_req()
    
    
